In [1]:
############# Mountiong Google Drive and Changing Working Directory PATH to Dataset Directory PATH ##################

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Selfie_Images/EX_Selfie&Selfie_Dataset.zip"

Archive:  /content/drive/MyDrive/Selfie_Images/EX_Selfie&Selfie_Dataset.zip
   creating: EX_Selfie&Selfie_Dataset/
   creating: EX_Selfie&Selfie_Dataset/code/
  inflating: EX_Selfie&Selfie_Dataset/code/DenseNet121.h5  
  inflating: EX_Selfie&Selfie_Dataset/code/DenseNet121_Selfie_Images.ipynb  
  inflating: EX_Selfie&Selfie_Dataset/code/inceptionV3.h5  
  inflating: EX_Selfie&Selfie_Dataset/code/inceptionV3_Selfie_Images.ipynb  
  inflating: EX_Selfie&Selfie_Dataset/code/MobileNet_V2.h5  
  inflating: EX_Selfie&Selfie_Dataset/code/MobileNet_V2_Selfie_Images(1).ipynb  
  inflating: EX_Selfie&Selfie_Dataset/code/ResNet50.h5  
  inflating: EX_Selfie&Selfie_Dataset/code/ResNet50_Selfie_Images.ipynb  
  inflating: EX_Selfie&Selfie_Dataset/code/VGG16.h5  
  inflating: EX_Selfie&Selfie_Dataset/code/VGG16_Selfie_Images.ipynb  
  inflating: EX_Selfie&Selfie_Dataset/code/VGG19.h5  
  inflating: EX_Selfie&Selfie_Dataset/code/VGG19_Selfie_Images_.ipynb  
   creating: EX_Selfie&Selfie_Dataset/EX_Se

In [3]:
directory_path = "/content/EX_Selfie&Selfie_Dataset"

################# Changing the Working Directory Path ################## 
import os
os.chdir(directory_path)
!ls

 code  'EX_Selfie&Selfie'   train_test_split


In [4]:
####################  importing libraries ######################

import numpy as np
import cv2
from scipy import ndimage
from skimage import io
from skimage.transform import rotate
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from skimage.color import rgb2hsv
import glob
import os
import keras
from skimage.transform import resize
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Model


In [5]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import splitfolders

In [7]:
splitfolders.ratio("/content/EX_Selfie&Selfie_Dataset/EX_Selfie&Selfie", output="/content/EX_Selfie&Selfie_Dataset/train_test_split",
    seed=42, ratio=(.8, .2), group_prefix=None) # default values

Copying files: 1318 files [00:00, 7180.28 files/s]


In [8]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as pi

In [9]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [10]:
train_path = '/content/EX_Selfie&Selfie_Dataset/train_test_split/train'
valid_path = '/content/EX_Selfie&Selfie_Dataset/train_test_split/val'

In [11]:
 # useful for getting number of classes
folders = glob('/content/EX_Selfie&Selfie_Dataset/train_test_split/train/*')

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('/content/EX_Selfie&Selfie_Dataset/train_test_split/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1054 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('/content/EX_Selfie&Selfie_Dataset/train_test_split/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 264 images belonging to 2 classes.


In [17]:
################### Loading Stored DenseNet121 Model #########################


model_path = "{}/code/DenseNet121.h5".format(directory_path)
DenseNet121_model = keras.models.load_model(model_path)


In [18]:
################### Loading Stored inceptionV3 Model #########################


model_path = "{}/code/inceptionV3.h5".format(directory_path)
inceptionV3_model = keras.models.load_model(model_path)


In [19]:
################### Loading Stored MobileNet_V2 Model #########################


model_path = "{}/code/MobileNet_V2.h5".format(directory_path)
MobileNet_V2_model = keras.models.load_model(model_path)


In [20]:

################### Loading Stored ResNet50 Model #########################

model_path = "{}/code/ResNet50.h5".format(directory_path)
ResNet50_model = keras.models.load_model(model_path)


In [21]:

################### Loading Stored VGG16 Model #########################

model_path = "{}/code/VGG16.h5".format(directory_path)
VGG16_model = keras.models.load_model(model_path)



In [22]:

################### Loading Stored VGG19 Model #########################

model_path = "{}/code/VGG19.h5".format(directory_path)
VGG19_model = keras.models.load_model(model_path)


In [23]:
#################### Defining Average Ensemble model from already trained 6 DNN Models

my_inputs = keras.Input(shape=(224, 224, 3,))
y1 = DenseNet121_model(my_inputs)
y2 = inceptionV3_model(my_inputs)
y3 = MobileNet_V2_model(my_inputs)
y4 = ResNet50_model(my_inputs)
y5 = VGG16_model(my_inputs)
y6 = VGG19_model(my_inputs)
print("The average of the layers in the model")
my_outputs = layers.average([y1, y2, y3, y4, y5, y6])
print("Ensemble model is being created")
ensemble_model = keras.Model(inputs=my_inputs, outputs=my_outputs)

print("Showing Ensembled Model Summary")
print(ensemble_model.summary())

The average of the layers in the model
Ensemble model is being created
Showing Ensembled Model Summary
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 DenseNet121_model (Functional)  (None, 2)           7137858     ['input_1[0][0]']                
                                                                                                  
 inceptionV3_model_2 (Functiona  (None, 2)           21905186    ['input_1[0][0]']                
 l)                                                                                       

In [24]:
################# Compiling Model and Evaluating on the test data ###############
ensemble_model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])
output = ensemble_model.evaluate(test_set)

9/9 [==============================] - 36s 944ms/step - loss: 0.1192 - accuracy: 0.9924
